## 井控微调数据集的处理

In [11]:
root_path = r'C:/Users/yanwei/Desktop/CQU/data'

In [13]:
import pandas as pd
import os

In [50]:
QA_path = os.path.join(root_path, 'book1_classified.xlsx')
data = pd.read_excel(QA_path, sheet_name='choice') # 读取选择题

In [51]:
data1 = data[['题目', '答案', '易错选项', 'Unnamed: 6', 'Unnamed: 7', '题目+答案']]
data1 = data1.rename(columns={'易错选项':'易错1', 'Unnamed: 6':'易错2', 'Unnamed: 7':'易错3', '题目+答案':'完整描述'})

In [52]:
# 将所有的nan 替换为“无”
data1 = data1.fillna('无')

In [ ]:
data1

In [56]:
# 将列名与每一行的值组成键值对
data1 = data1.to_dict(orient='records')

In [55]:
import json

In [58]:
with open(os.path.join(root_path, 'convert_kv_format.json'), 'w', encoding='utf-8') as f:
    json.dump(data1, f, ensure_ascii=False, indent=4)

In [ ]:
!pip install ZhipuAI

**智谱API调用**

In [ ]:
# 同步调用
from zhipuai import ZhipuAI  # 先安装ZhipuAI的包 pip install ZhipuAI
client = ZhipuAI(api_key="d96b3dc3c1b2ce95e379396f4d5d6da1.B55pm2il4dXs8822") # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
    # messages是json格式的数据，大模型逐条响应
        {"role": "user", "content": "作为一名营销专家，请为我的产品创作一个吸引人的slogan"},
        # {"role": "assistant", "content": "当然，为了创作一个吸引人的slogan，请告诉我一些关于您产品的信息"},
        # {"role": "user", "content": "智谱AI开放平台"},
        # {"role": "assistant", "content": "智启未来，谱绘无限一智谱AI，让创新触手可及!"},
        # {"role": "user", "content": "创造一个更精准、吸引人的slogan"}
    ],
)
# 直接输出response，查看响应的具体内容
# print(response)
print(response.choices[0].message.content) # 


In [ ]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key='d96b3dc3c1b2ce95e379396f4d5d6da1.B55pm2il4dXs8822')

while True:
    prompt = input('\n user: ')
    response = client.chat.completions.create(
        model="glm-4",
        messages = [
            {'role': 'user', 'content': prompt}
        ],
    )
    answer = response.choices[0].message.content
    print('\n answer: ', answer)

In [ ]:
# 异步调用
import time
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="d96b3dc3c1b2ce95e379396f4d5d6da1.B55pm2il4dXs8822") # 请填写您自己的APIKey

response = client.chat.asyncCompletions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {
            "role": "user",
            "content": "请你作为童话故事大王，写一篇短篇童话故事，故事的主题是要永远保持一颗善良的心，要能够激发儿童的学习兴趣和想象力，同时也能够帮助儿童更好地理解和接受故事中所蕴含的道理和价值观。"
        }
    ],
)
# 获取响应ID
task_id = response.id
task_status = ''
get_cnt = 0

while task_status != 'SUCCESS' and task_status != 'FAILED' and get_cnt <= 40:
    # 查询响应结果
    result_response = client.chat.asyncCompletions.retrieve_completion_result(id=task_id)
    print(result_response)
    task_status = result_response.task_status

    time.sleep(2)
    get_cnt += 1


In [ ]:
## zhipu 文生图大模型调用

from zhipuai import ZhipuAI
client = ZhipuAI(api_key='d96b3dc3c1b2ce95e379396f4d5d6da1.B55pm2il4dXs8822')

response = client.images.generations(
    model='cogview-3',
    prompt='一只小狗'
)
print(response.data[0].url)

**Qwen API调用**

In [ ]:
# 单论对话
import random
from http import HTTPStatus
import dashscope

dashscope.api_key = 'sk-029ca2483e6749a0aceff839174a9945'

def call_with_messages():
    prompt = input("user:")
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}]
    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_turbo, # 选择模型
        messages=messages,
        # set the random seed, optional, default to 1234 if not set
        seed=random.randint(1, 10000),
        result_format='message',  # set the result to be "message" format.
    )
    # 之后HTTPStatus为OK时，才是调用成功
    if response.status_code == HTTPStatus.OK:
        print(response)
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))

def call_with_prompt():
    prompt = input("user:")
    response = dashscope.Generation.call(
        model=dashscope.Generation.Models.qwen_turbo, # 选择模型
        prompt=prompt
    )
    # The response status_code is HTTPStatus.OK indicate success,
    # otherwise indicate request is failed, you can get error code
    # and message from code and message.
    if response.status_code == HTTPStatus.OK:
        print(response.output)  # The output text
        print(response.usage)  # The usage information
    else:
        print(response.code)  # The error code.
        print(response.message)  # The error message.

if __name__ == '__main__':

    call_with_messages()
    print("\n")
    call_with_prompt()

In [ ]:
## 多轮对话调用
from http import HTTPStatus
import dashscope
from dashscope import Generation
from dashscope.api_entities.dashscope_response import Role

dashscope.api_key = 'sk-029ca2483e6749a0aceff839174a9945' # 设置API_KEY

def conversation_with_messages():
    messages = [{'role': Role.SYSTEM, 'content': 'You are a helpful assistant.'}  ]
    # 循环实现多轮会话
    while True:
        prompt = input("USER:")
        # 添加新一轮会话用户的问题
        messages.append({'role': Role.USER, 'content': prompt})
        response = Generation.call(
            Generation.Models.qwen_turbo, #选择响应的模型
            messages=messages,
            result_format='message',  # set the result to be "message" format.
        )
        if response.status_code == HTTPStatus.OK:
            print(response)
            # 把模型的输出添加到messages中
            messages.append({'role': response.output.choices[0]['message']['role'],
                             'content': response.output.choices[0]['message']['content']})
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            exit()

if __name__ == '__main__':
    conversation_with_messages()


In [ ]:
# 通过openai 进行调用

from openai import OpenAI

try:
    client = OpenAI(
        api_key='sk-029ca2483e6749a0aceff839174a9945',
        base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
    )
    
    completion = client.completions.create(
        model='qwen-turbo',
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': '你是谁？'}
        ]
    )
    print(completion.choices[0].message.content)
except Exception as e:
    print(f'错误信息：{e}')

**Ollama 调用本地大模型**

In [ ]:
import ollama
res=ollama.chat(model="qwen2.5:7b",stream=False,messages=[{"role": "user","content": "你是谁"}],options={"temperature":0})
print(res)

In [20]:
import random
from http import HTTPStatus
import dashscope

dashscope.api_key = 'sk-029ca2483e6749a0aceff839174a9945'

def call_with_messages(prompt):
    # prompt = input("user:")
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}]
    response = dashscope.Generation.call(
        dashscope.Generation.Models.qwen_turbo, # 选择模型
        messages=messages,
        # set the random seed, optional, default to 1234 if not set
        seed=random.randint(1, 10000),
        result_format='message',  # set the result to be "message" format.
    )
    # 之后HTTPStatus为OK时，才是调用成功
    if response.status_code == HTTPStatus.OK:
        # print(response)
        return response.output.choices[0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))


def call_with_prompt(prompt):
    # prompt = input("user:")
    response = dashscope.Generation.call(
        model=dashscope.Generation.Models.qwen_turbo, # 选择模型
        prompt=prompt
    )
    # The response status_code is HTTPStatus.OK indicate success,
    # otherwise indicate request is failed, you can get error code
    # and message from code and message.
    if response.status_code == HTTPStatus.OK:
        return response.output.text
        # print(response.output)  # The output text
        # print(response.usage)  # The usage information
    else:
        print(response.code)  # The error code.
        print(response.message)  # The error message.

if __name__ == '__main__':

    prompt = input('user: ')
    # print(call_with_messages(prompt))
    # print("\n")
    print(call_with_prompt(prompt))

你好！很高兴能为你提供帮助。有什么问题我可以回答吗？
